<a href="https://colab.research.google.com/github/yuktaX/ReedSolomon/blob/main/NT_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Team Members**
1. Yukta Arvind Rajapur - IMT2021066
2. Brij Bidhin Desai - IMT2021067

# Reed Solomon Code Implementaion
The following code implements Reed Solomans code for message trasmission and recovery. It uses intermediate functions such as Binary EGCD, and CRT. Before running the main code, make sure to run all the cells before it. The following cells are:
1. pip installs - Installs needed libraries
2. Intermediate functions - Binary EGCD, CRT, GlobalSetup(), Trasmit(), ReedSolomonSend(), ReedSolomonReceive()
3. User-input Main - If you want to execute your own custom testcase, run this cell
4. Executable main - Run this cell if you want to test a predefined testcase below
5. Testcase cells - Run whichever predefined case you want to test

In [1]:
pip install gmpy2            

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from functools import reduce
import random
from math import floor

def divisible_by_2(x):
  return x % 2 == 0 

def binaryEGCD(a,b):
  r1, r2, e = a, b, 0
  while divisible_by_2(r1) and divisible_by_2(r2):
    r1 = r1 >> 1
    r2 = r2 >> 1
    e += 1

  a1, b1, s0, t0, s1, t1 = r1, r2, 1, 0, 0, 1

  while(r2 > 0):

    while divisible_by_2(r1):
      r1 = r1 >> 1
      if divisible_by_2(s0) and divisible_by_2(t0):
        s0 = s0 >> 1
        t0 = t0 >> 1
      else:
        s0, t0 = (s0 + b1) >> 1, (t0 - a1) >> 1

    while divisible_by_2(r2):
      r2 = r2 >> 1
      if divisible_by_2(s1) and divisible_by_2(t1):
        s1 = s1 >> 1
        t1 = t1 >> 1
      else:
        s1, t1 = (s1 + b1) >> 1, (t1 - a1) >> 1
    
    if r2 < r1: #swap
      r1, s0, t0, r2, s1, t1 = r2, s1, t1, r1, s0, t0

    r2 -= r1
    s1 -= s0
    t1 -= t0

  d = (2**e)*r1
  egcd = [d, s0, t0]
  return egcd


def EGCD(a, b, complete = True, r_star = 0):
  r, r1 = a, b
  s0, s1, t0, t1 = 1, 0, 0, 1
  while r1 != 0:
    q = r // r1
    r2 = r % r1
    r, s0, t0, r1, s1, t1 = r1, s1, t1, r2, s0 - s1*q, t0 - t1*q

    #if we need specific r', s', t'
    if complete == False and r <= r_star:
      return [r, s0, t0]

  d = r
  return [d, s0, t0]

def crtEncode(a, N, k):
  aList = []
  for i in range(k):
    aList.append(a % N[i])
  return aList

def crtDecode(A, N, k):
  n = mpz(reduce((lambda x, y: x * y), N))
  E = []
  a = 0
  for i in range (k):
    n_tmp = mpz(n//N[i])
    b_tmp = mpz(n_tmp % N[i])
    t_tmp = binaryEGCD(N[i], b_tmp)[2]
    if t_tmp >= 0:
      t_tmp = t_tmp
    else:
      t_tmp = t_tmp + N[i]
    E.append((n_tmp*t_tmp))
  for i in range(k):
    a += A[i]*E[i]
  a = a % n
  return a

def GlobalSetup(u, M):
  # we want to find k primes whose product n satisfies n > 2MP^2
  
  # assuming the largest prime available is 2^8, P = (2^8)^l = (2^8)^(u*k)
  
  # Hence, the value of P changes every time a prime is found in the
  # while loop
  
  # P_sq = (2^16)^l = (2^16)^(u*k)
  
  # initially we will take P_sq to be (2^16)^(u) and keep multiplying it
  # with cutoff everytime a prime is found in the while loop
  
  primes = []
  P_sq = mpfr(2**(32*u))
  initial_val = mpz(2**16 - 1)
  cutoff = mpz(2*M)
  product = mpz(1)
  
  while (cutoff >= product):
    if initial_val.is_prime() == True:
      primes.append(initial_val)
      product *= initial_val
      cutoff *= P_sq

    initial_val -= 2
   
  return primes

def Transmit(A):
  k = len(primes)
  l = random.randint(0, int(u*k))
  bad_indices = [0]*k
  count = 0
  while (count <= l):
    random_index = random.randint(0, k-1)
    if bad_indices[random_index] == 0:
      bad_indices[random_index] = 1
      count += 1
    
  B = []
  for i in range(k):
    if bad_indices[i] == 1:
      new_val = random.randint(0, primes[i] - 1)
      while new_val == A[i]:
        new_val = random.randint(0, primes[i] - 1)
      B.append(new_val)
    else:
      B.append(A[i])
  return B

def ReedSolomonSend(a):
  A = crtEncode(a, primes, k)
  return Transmit(A)

def ReedSolomonReceive(b):
  global primes, k, P, M
  msg_rcv = b
  msg_reconstructed = crtDecode(msg_rcv, primes, len(primes))
  n = reduce((lambda x, y: x * y), primes)
  res = EGCD(n, msg_reconstructed, False, M*P)
  r_dash, s_dash, t_dash = res[0], res[1], res[2]
  if r_dash % t_dash == 0:
    print("Success")
    print("Message receieved = ", r_dash // t_dash)
  else:
    print("Error, recovery failed")
    print("Message received = ", msg_reconstructed)

In [48]:
#main-user input based
from gmpy2 import mpz, mpfr, is_prime

length_of_msg = int(input("Enter the number of bits of the message: "))
if length_of_msg <= 0:
  print("It has to be a positive number")
  exit()
#Set M = 2**l as upper bound
M = mpz(1 << length_of_msg)

while(1):
  a = int(input("Enter message a: "))
  if a > M:
    print("a has to be smaller than ", M)
  else:
    break
    
while(1):
  u = float(input("Enter corruption factor(u), a value 0<u<1: "))
  if u < 0 or u > 1:
    print("It has to be a fractional value between 0 and 1")
  else:
    break

primes = GlobalSetup(u, M)
k = len(primes)
l = floor(u*k)
P_arr = sorted(primes, reverse = True)
P = mpz(1)
for i in range(l):
  P = P*P_arr[i]
#print("P: ", P)
ReedSolomonReceive(ReedSolomonSend(a))


Enter the number of bits of the message: 1000
Enter message a: 2479515631440564111848484445589631447856582658641244966555997783002587
Enter corruption factor(u), a value 0<u<1: 0.1
Success
Message receieved =  2479515631440564111848484445589631447856582658641244966555997783002587


In [47]:
#main-executable
from gmpy2 import mpz, mpfr, is_prime

def main(length_of_msg, a, u):
  print("Message sent:", a)
  M = mpz(1 << length_of_msg)
  primes = GlobalSetup(u, M)
  k = len(primes)
  l = floor(u*k)
  P_arr = sorted(primes, reverse = True)
  P = mpz(1)
  for i in range(l):
    P = P*P_arr[i]
  #print("P: ", P)
  ReedSolomonReceive(ReedSolomonSend(a))

In [29]:
#Testcase 1
length_of_msg = 16
u = 0.1
a = 9943
main(length_of_msg, a, u)

Message sent: 9943
Error, recovery failed
Message received =  3520018218


In [30]:
#Testcase 2
length_of_msg = 16
u = 0.2
a = 55889
main(length_of_msg, a, u)

Message sent: 55889
Error, recovery failed
Message received =  3029091719


In [31]:
#Testcase 3
length_of_msg = 20
u = 0.4
a = 818153
main(length_of_msg, a, u)

Message sent: 818153
Error, recovery failed
Message received =  3223763282


In [32]:
#Testcase 4
length_of_msg = 50
u = 0.4
a = 714401401184811
main(length_of_msg, a, u)

Message sent: 714401401184811
Error, recovery failed
Message received =  4176236434


In [36]:
#Testcase 5
length_of_msg = 100
u = 0.1
a = 314405641118484844
main(length_of_msg, a, u)

Message sent: 314405641118484844
Success
Message receieved =  314405641118484844


In [49]:
#Testcase 6
length_of_msg = 1000
u = 0.1
a = 2479515631440564111848484445589631447856582658641244966555997783002587
main(length_of_msg, a, u)

Message sent: 2479515631440564111848484445589631447856582658641244966555997783002587
Success
Message receieved =  2479515631440564111848484445589631447856582658641244966555997783002587
